##  **Giới thiệu**

Trong bài này, chúng ta sử dụng PyTorch để xây dựng một mạng CNN đơn giản. Sau đó chúng ta sẽ huấn luyện và đánh giá model với tập dữ liệu MNIST nhé.

## Lưu ý về cách làm bài tập
Các bạn điền vào phần **None** và các đoạn code đã được ẩn đi.


### Tổng quan một mạng CNN cơ bản

![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)

### MNIST dataset

Trong bài tập này, chúng ta sẽ sử dựng tập MNIST rất nổi tiếng vể  các chữ số viết tay từ 0->9. Tập dataset này bao gồm 60000 ảnh cho training và 10000 ảnh cho testing. Các bức ảnh này đều đã được căn giữa và chỉnh với kích thước cố định là 28x28.

Trong phần tiền xử lý, chúng ta sẽ cần chuẩn hóa các giá trị pixel của mỗi ảnh về khoảng [0,1], kiểu dữ liệu sẽ là float32

<!-- ![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png) -->

Chi tiết tại: http://yann.lecun.com/exdb/mnist/

## **Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import random

from torchsummary import summary

## **Setup**

- Chúng ta sẽ setup một số hyper-parameters cũng như một số giá trị cần dùng theo hướng dẫn nhé
- Ở đây, các bạn vào Runtime, chọn Change the runtime type và chọn GPU nhé.

### Hyperparameters

In [ ]:
# Số classes trong tập MNIST
num_classes = None

# Số epoch
epochs = None

# Các tham số cần thiết trong quá trình training
learning_rate = 0.001
batch_size = 128
display_step = 100

# Model path
checkpoint = 'model.pth'

# device: cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Download MNIST dataset in local system

In [ ]:
# Định nghĩa tham số transform
transform=transforms.Compose([
    transforms.ToTensor(), # Chuyển ảnh sang dạng Tensor
    transforms.Normalize((0.5,), (0.5,)) # Normalize ảnh với mean và standard deviation là 0.5
    ])

# Load MNIST dataset từ torchvision.datasets
train_data = datasets.MNIST(
    root='data', 
    train=True, 
    transform=transform, 
    download=True
)
test_data = datasets.MNIST(
    root='data', 
    train=False, 
    transform=transform,
)


In [ ]:
print(train_data)
print(test_data)

In [ ]:
print(train_data.data.size())
print(train_data.targets.size())

**Visualization of MNIST dataset**

In [ ]:
figure = plt.figure(figsize=(10, 8))
cols, rows = 5, 5
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

### Preparing data for training with DataLoaders
- Để tiện cho việc xử lý dữ dữ liệu vào các batches cũng như reshuffle dữ liệu qua mỗi epoch thì chúng ta sẽ sử dụng hàm sẵn có của PyTorch là [DataLoader](https://pytorch.org/docs/stable/data.html) 

In [ ]:
train_loader = torch.utils.data.DataLoader(None, batch_size=None)
test_loader = torch.utils.data.DataLoader(None, batch_size=None)

## **Model**

- Trong bài này, chúng ta sẽ định nghĩa một class Net để xây dựng một model có cấu trúc như hình ở đầu notebook. Tuy nhiên, ở đây để tránh khả năng bị overfitting của mô hình thì layer `Dropout` sẽ được thêm vào. Mô hình sẽ có cấu trúc như sau: `3x(Conv2d -> ReLU) -> MaxPool -> Dropout -> Flatten -> 2x(Linear ->  ReLU) -> Linear`
- Kích thước ảnh đầu vào: (28, 28, 1)
- Ở đây, các layer `Conv2d` có các thông số lần lượt là: `filter = (32, 64, 64); kernel_size = 3; stride = 1; valid padding`
- Các layer `Linear` lần lượt số node là `128, 64, num_classes`
- Sau layer `MaxPool2d` thì `height` và `width` của ảnh sẽ giảm đi một nửa
- Hệ số layer `Dropout` = 0.5
- Tham khảo: [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), [MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html), [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html), [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html), [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 3x(Conv2d -> ReLU) -> MaxPool -> Dropout -> Flatten -> 2x(Linear ->  ReLU) -> Linear
        self.relu = None
        self.dropout = None
        self.flatten = None
        self.conv1 = nn.Conv2d(in_channels=None, out_channels=None, kernel_size=None, stride=None, padding=None)
        self.conv2 = nn.Conv2d(in_channels=None, out_channels=None, kernel_size=None, stride=None, padding=None)
        self.conv3 = nn.Conv2d(in_channels=None, out_channels=None, kernel_size=None, stride=None, padding=None)
        self.maxpool = nn.MaxPool2d(kernel_size=None, stride=None)
        self.fc1 = nn.Linear(in_features=None, out_features=None)
        self.fc2 = nn.Linear(in_features=None, out_features=None)
        self.fc3 = nn.Linear(in_features=None, out_features=None)

    def forward(self, x):
        ### START CODE HEAR ≈ 9-18 lines
        ## 3x(Conv2d -> ReLU) -> MaxPool -> Dropout -> Flatten -> 2x(Linear ->  ReLU) -> Linear


        ### END CODE HERE
        return x

In [ ]:
# Load model vào GPU, ví dụ: input = model().to(device)
model = None
summary(model, (1, 32, 32))

## **Training phase**



In [ ]:
# Define loss and optimizer
criterion = None # CrossEntropy
optimizer = None # Adam Optimizer set params=model.parameters(), lr=learning_rate
best_val_loss = 999

# Loop for each epoch
for epoch in range(1, epochs + 1):

    # Quá trình training 
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Load dữ liệu vào GPU
        data, target = None, None

        # Clear gradients cho mỗi batch
        None # zero grad
        output = model(data)

        # Backpropagation, tính gradients
        loss = criterion(output, target)
        None #backward

        # Apply gradients để update lại tham số
        None
        if batch_idx % display_step == 0:
            print('Train Epoch {}: [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    # Quá trình testing 
    model.eval()
    test_loss = 0
    correct = 0

    # Set no grad cho quá trình testing
    with torch.no_grad():
        for data, target in test_loader:
            # Load dữ liệu vào GPU
            data, target = None, None
            output = model(data)
            output = None # sử dụng hàm log_sotmax của pytorch để tính xác suất cho output, dim = 1
            test_loss += criterion(output, target)
            pred = None # Sử dụng hàm argmax để lấy predicted label, chú ý keepdim=True
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(None) 
    if test_loss < best_val_loss:
      best_val_loss = test_loss
      torch.save(model.state_dict(), checkpoint)  # Lưu model path
      print("***********    TEST_ACC = {:.2f}%    ***********".format(correct/100))

In [ ]:
# Load lại model đã train
model.load_state_dict(torch.load(checkpoint))

# Xem lại thông số của model 
model.eval()

## **Prediction**

In [ ]:
# Lấy ra một batch trong tập test
item = iter(test_loader)
data, target = item.next()

# Lấy random index của một phần tử trong batch đó
test_idx = random.choice(range(len(data)))

# Lấy một ví dụ trong tập test
data = data[test_idx]
target = target[test_idx]
assert data.shape == (1, 28, 28)

In [ ]:
# Predict sử dụng model đã train
def plot(data, model):
  data = torch.unsqueeze(data, dim=0) # unsqueeze data
  data = None # Đưa data vào GPU
  output = model(data)
  output = None # Tính xác suất từng class của output, sử dụng log_softmax, dim = 1
  pred = None # Lấy class có xác suất cao nhất, keep dim = True, sử dụng argmax
  print("Predict Number : ", pred[0][0].detach().cpu().numpy()) 
  plt.imshow(data[0][0].detach().cpu().numpy(), cmap='gray')
  plt.show()

In [ ]:
plot(data, model)

## More Exercises

### Exercise 1: VGG-like model
* Implement a simplified VGG model by building 3 'blocks' of 2 convolutional layers each
* Do MaxPooling after each block
* The first block should use at least 32 filters, later blocks should use more
* You can use 3x3 filters
* Use zero-padding to be able to build a deeper model (see the `padding` attribute)
* Use a dense layer with at least 128 hidden nodes.
* You can use ReLU activations everywhere (where it makes sense)
* Plot and interpret the learning curves

### Exercise 2: Regularization
* Explore different ways to regularize your VGG-like model
  * Try adding some dropout after every MaxPooling and Dense layer.
    * What are good Dropout rates? Try a fixed Dropout rate, or increase the rates in the deeper layers.
  * Try batch normalization together with Dropout
    * Think about where batch normalization would make sense 
* Plot and interpret the learning curves
